In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os
import keras
from keras.layers import Dense,Conv2D, BatchNormalization, Activation,Dropout
from keras.layers import AveragePooling2D, Input, Flatten,GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
import keras.backend as K
import cv2
import seaborn as sns
import PIL
import skimage
from keras.models import Model
%matplotlib inline

Using TensorFlow backend.


In [3]:
def data_generate(data_path, batch_size, argumentation = True, shuffle=False):
    while True:
        fid = h5py.File(data_path, 'r')
        data_len = fid['sen1'].shape[0]
        c = [i for i in range(int(data_len/batch_size) + 1)]
        if shuffle:
            np.random.shuffle(c)
        for i in c:
            t = np.random.rand()
            try:
                y_b = np.array((fid['label'][i * batch_size: (i+1)*batch_size]))
                x_b1 = fid['sen1'][i * batch_size: (i+1)*batch_size]
                x_b2 = fid['sen2'][i * batch_size: (i+1)*batch_size]
                x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                if argumentation:
                    if t < 0.6:
                        k = np.random.choice([1,2,3,4,5,6])
                        if k == 2:
                            x_b1 = np.rot90(x_b1, k=1, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=1, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=1, axes=(1,2))
                        elif k == 3:
                            x_b1 = np.rot90(x_b1, k=2, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=2, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=2, axes=(1,2))
                        elif k == 4:
                            x_b1 = np.rot90(x_b1, k=3, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=3, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=3, axes=(1,2))
                        elif k == 5:
                            x_b1 = x_b1[:,::-1,:,:]
                            x_b2 = x_b2[:,::-1,:,:]
                            x_b3 = x_b3[:,::-1,:,:]
                        elif k == 6:
                            x_b1 = x_b1[:,:,::-1,:]
                            x_b2 = x_b2[:,:,::-1,:]
                            x_b3 = x_b3[:,:,::-1,:]
                yield ([x_b1, x_b2, x_b3], y_b)
            except:
                x_b1 = fid['sen1'][i*batch_size:]
                x_b2 = fid['sen2'][i*batch_size:]
                x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                y_b = np.array((fid['label'][i * batch_size:]))
                if argumentation:
                    if t < 0.6:
                        k = np.random.choice([2,3,4,5,6])
                        
                        if k == 2:
                            x_b1 = np.rot90(x_b1, k=1, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=1, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=1, axes=(1,2))
                        elif k == 3:
                            x_b1 = np.rot90(x_b1, k=2, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=2, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=2, axes=(1,2))
                        elif k == 4:
                            x_b1 = np.rot90(x_b1, k=3, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=3, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=3, axes=(1,2))
                        elif k == 5:
                            x_b1 = x_b1[:,::-1,:,:]
                            x_b2 = x_b2[:,::-1,:,:]
                            x_b3 = x_b3[:,::-1,:,:]
                        elif k == 6:
                            x_b1 = x_b1[:,:,::-1,:]
                            x_b2 = x_b2[:,:,::-1,:]
                            x_b3 = x_b3[:,:,::-1,:]
                yield ([x_b1, x_b2, x_b3] , y_b)

In [4]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [6]:
def resnet_v2(input_tensor, depth, num_classes=17):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    #inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=input_tensor,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    #y = Flatten()(x)
    #outputs = Dense(num_classes,
     #               activation='softmax',
     #               kernel_initializer='he_normal')(y)

    # Instantiate model.
    #model = Model(inputs=inputs, outputs=outputs)
    return x

In [7]:
import keras.backend as K
import tensorflow as tf
def focal_loss(classes_num, gamma=2., alpha=.25, e=0.1):
    # classes_num contains sample number of each classes
    def focal_loss_fixed(target_tensor, prediction_tensor):
        '''
        prediction_tensor is the output tensor with shape [None, 100], where 100 is the number of classes
        target_tensor is the label tensor, same shape as predcition_tensor
        '''
        import tensorflow as tf
        from tensorflow.python.ops import array_ops
        from keras import backend as K

        #1# get focal loss with no balanced weight which presented in paper function (4)
        zeros = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)
        one_minus_p = array_ops.where(tf.greater(target_tensor,zeros), target_tensor - prediction_tensor, zeros)
        FT = -1 * (one_minus_p ** gamma) * tf.log(tf.clip_by_value(prediction_tensor, 1e-8, 1.0))

        #2# get balanced weight alpha
        classes_weight = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)

        total_num = float(sum(classes_num))
        classes_w_t1 = [ total_num / ff for ff in classes_num ]
        sum_ = sum(classes_w_t1)
        classes_w_t2 = [ ff/sum_ for ff in classes_w_t1 ]   #scale
        classes_w_tensor = tf.convert_to_tensor(classes_w_t2, dtype=prediction_tensor.dtype)
        classes_weight += classes_w_tensor

        alpha = array_ops.where(tf.greater(target_tensor, zeros), classes_weight, zeros)

        #3# get balanced focal loss
        balanced_fl = alpha * FT
        balanced_fl = tf.reduce_sum(balanced_fl)

        #4# add other op to prevent overfit
        # reference : https://spaces.ac.cn/archives/4493
        nb_classes = len(classes_num)
        fianal_loss = (1-e) * balanced_fl + e * K.categorical_crossentropy(K.ones_like(prediction_tensor)/nb_classes, prediction_tensor)

        return fianal_loss
    return focal_loss_fixed

In [8]:
input_shape1 = (32, 32, 8)
input_shape2 = (32, 32, 10)
input_shape3 = (32, 32, 18)
input_1 = Input(shape=input_shape1)
input_2 = Input(shape=input_shape2)
input_3 = Input(shape=input_shape3)
L1 = BatchNormalization()(input_1)
L2 = BatchNormalization()(input_2)
L3 = BatchNormalization()(input_3)
L1 = resnet_v2(input_tensor=L1, depth=56)
L2 = resnet_v2(input_tensor=L2, depth=56)
L3 = resnet_v2(input_tensor=L3, depth=56)

#L1 = Conv2D(16, kernel_size=3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(L1)
#L2 = Conv2D(16, kernel_size=3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(L2)


L = keras.layers.concatenate([L1, L2, L3])
L = Dense(512, activation='relu')(L)
L = BatchNormalization()(L)
L = Dropout(0.5)(L)
L = Dense(17, activation='softmax', kernel_initializer='he_normal')(L)
model = Model(inputs=[input_1, input_2, input_3], outputs=L)
class_nums = [5068, 24431, 31693,  8651, 16493, 35290,  3269, 39326,
       13584, 11954, 42902,  9514,  9165, 41377,  2392,  7898,
       49359]
#model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.compile(loss=focal_loss(class_nums), optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [9]:
save_dir = './models'
model_name = 'LCZ_model_3_inputsss-950-loss-f-56.h5'
filepath = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc',save_best_only=True, save_weights_only=True)
lr_reducr = ReduceLROnPlateau(factor=0.1,cooldown=0, patience=3, min_lr=1e-6)
early = EarlyStopping(monitor='val_acc',patience=5)
calls = [checkpoint, lr_reducr, early]

In [10]:
train_gen = data_generate('./training.h5', batch_size=128, argumentation=True, shuffle=True)
val_gen = data_generate('./validation.h5', batch_size=128, argumentation=True,shuffle=False)

In [11]:
model.fit_generator(train_gen, steps_per_epoch=352366//128+1, epochs=200,  verbose=1,callbacks=calls, validation_data=val_gen,
                   validation_steps=24119//128+1)

Epoch 1/200
2753/2753 [==============================] - 1627s 591ms/step - loss: 3.8941 - acc: 0.6983 - val_loss: 5.2324 - val_acc: 0.5471
Epoch 2/200
2753/2753 [==============================] - 1597s 580ms/step - loss: 2.2280 - acc: 0.7791 - val_loss: 4.8957 - val_acc: 0.5273
Epoch 3/200
2753/2753 [==============================] - 1576s 572ms/step - loss: 1.8555 - acc: 0.8062 - val_loss: 5.3669 - val_acc: 0.5602
Epoch 4/200
2753/2753 [==============================] - 1573s 571ms/step - loss: 1.6736 - acc: 0.8244 - val_loss: 5.6762 - val_acc: 0.5704
Epoch 5/200
2753/2753 [==============================] - 1565s 568ms/step - loss: 1.5526 - acc: 0.8365 - val_loss: 6.2178 - val_acc: 0.5735
Epoch 6/200
2753/2753 [==============================] - 1585s 576ms/step - loss: 1.4697 - acc: 0.8469 - val_loss: 5.4164 - val_acc: 0.5348
Epoch 7/200
2753/2753 [==============================] - 1565s 569ms/step - loss: 1.2409 - acc: 0.8833 - val_loss: 5.2513 - val_acc: 0.6136
Epoch 8/200
2753/275

In [12]:
def data_generate2(data1, data2, data3, batch_size, argumentation = True, shuffle=False):
    while True:
        data_len = data1.shape[0]
        c = [i for i in range(int(data_len/batch_size) + 1)]
        if shuffle:
            np.random.shuffle(c)
        for i in c:
            t = np.random.rand()
            try:
                y_b = data3[i*batch_size:(i+1)*batch_size]
                x_b1 = data1[i * batch_size: (i+1)*batch_size]
                x_b2 = data2[i * batch_size: (i+1)*batch_size]
                x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                if argumentation:
                    if t < 0.6:
                        k = np.random.choice([2,3,4,5,6])
                        if k == 2:
                            x_b1 = np.rot90(x_b1, k=1, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=1, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=1, axes=(1,2))
                        elif k == 3:
                            x_b1 = np.rot90(x_b1, k=2, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=2, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=2, axes=(1,2))
                        elif k == 4:
                            x_b1 = np.rot90(x_b1, k=3, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=3, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=3, axes=(1,2))
                        elif k == 5:
                            x_b1 = x_b1[:,::-1,:,:]
                            x_b2 = x_b2[:,::-1,:,:]
                            x_b3 = x_b3[:,::-1,:,:]
                        elif k == 6:
                            x_b1 = x_b1[:,:,::-1,:]
                            x_b2 = x_b2[:,:,::-1,:]
                            x_b3 = x_b3[:,:,::-1,:]
                yield ([x_b1, x_b2, x_b3], y_b)
            except:
                y_b = data3[i*batch_size:]
                x_b1 = data1[i*batch_size:]
                x_b2 = data2[i*batch_size:]
                x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                if argumentation:
                    if t < 0.6:
                        k = np.random.choice([2,3,4,5,6])
                        if k == 2:
                            x_b1 = np.rot90(x_b1, k=1, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=1, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=1, axes=(1,2))
                        elif k == 3:
                            x_b1 = np.rot90(x_b1, k=2, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=2, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=2, axes=(1,2))
                        elif k == 4:
                            x_b1 = np.rot90(x_b1, k=3, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=3, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=3, axes=(1,2))
                        elif k == 5:
                            x_b1 = x_b1[:,::-1,:,:]
                            x_b2 = x_b2[:,::-1,:,:]
                            x_b3 = x_b3[:,::-1,:,:]
                        elif k == 6:
                            x_b1 = x_b1[:,:,::-1,:]
                            x_b2 = x_b2[:,:,::-1,:]
                            x_b3 = x_b3[:,:,::-1,:]
                yield ([x_b1, x_b2, x_b3], y_b)

In [13]:
valid = h5py.File('./validation.h5', 'r')
valid_s1 = valid['sen1']
valid_s2 = valid['sen2']
valid_label = valid['label']
valid_s1 = np.array(valid_s1)
valid_s2 = np.array(valid_s2)
valid_label = np.array(valid_label)

In [14]:
valid_s1.shape,valid_s2.shape,valid_label.shape

((24119, 32, 32, 8), (24119, 32, 32, 10), (24119, 17))

In [15]:
from sklearn.model_selection import StratifiedKFold, KFold
skf = StratifiedKFold(n_splits=5, random_state=2018)
kf = KFold(n_splits=5, random_state=2018)

In [16]:
y = np.argmax(valid_label, axis=1)

In [17]:
test = h5py.File('./round2_test_a_20190121.h5')
test_s1 = np.array(test['sen1'])
test_s2 = np.array(test['sen2'])

In [18]:
24119*0.8,24119*0.2

(19295.2, 4823.8)

In [19]:
pred = np.zeros((test_s1.shape[0], 17))
test_s3 = np.concatenate([test_s1, test_s2], axis=-1)
for i ,(tr_ind, val_ind) in enumerate(skf.split(valid_s1, y)):
    tr1, tr2, trl = valid_s1[tr_ind], valid_s2[tr_ind], valid_label[tr_ind]
    va1, va2, val = valid_s1[val_ind], valid_s2[val_ind], valid_label[val_ind]
    save_dir = './model_2/'
    model_name = 'LCZ_model_FOLD_3_input_%s_loss-f-56.h5' % str(i)
    filepath = os.path.join(save_dir, model_name)
    checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc',save_best_only=True, save_weights_only=True)
    lr_reducr = ReduceLROnPlateau(factor=0.1,cooldown=0, patience=3, min_lr=1e-6)
    early = EarlyStopping(monitor='val_acc',patience=5,)
    calls = [checkpoint, lr_reducr, early]
    class_num = [ 256., 1254., 2353.,  849.,  757., 1906.,  474., 3395., 1914.,
        860., 2287.,  382., 1202., 2747.,  202.,  672., 2609.]
    class_num = [int(x*0.8) for x in class_num]
    model.compile(loss=focal_loss(class_nums), optimizer=Adam(lr=0.0001), metrics=['accuracy'])
    model.load_weights('./models/LCZ_model_3_inputsss-950-loss-f-56.h5')
    tr_gen = data_generate2(tr1, tr2, trl,argumentation=True,batch_size=128,shuffle=True)
    va_gen = data_generate2(va1, va2, val,argumentation=True,batch_size=128,shuffle=False)
    model.fit_generator(tr_gen, steps_per_epoch=19295//128+1, epochs=200, validation_data=va_gen,validation_steps=4823//128+1, callbacks=calls)
    model.load_weights(filepath)
    for j in range(4):
        te1 = np.rot90(test_s1,k=j,axes=(1,2))
        te2 = np.rot90(test_s2,k=j,axes=(1,2))
        te3 = np.rot90(test_s3,k=j,axes=(1,2))
        pred += model.predict([te1, te2, te3])
    te1 = test_s1[:,::-1,:,:]
    te2 = test_s2[:,::-1,:,:]
    te3 = test_s3[:,::-1,:,:]
    pred += model.predict([te1, te2, te3])
    te1 = test_s1[:,:,::-1,:]
    te2 = test_s2[:,:,::-1,:]
    te3 = test_s3[:,:,::-1,:]
    pred += model.predict([te1, te2, te3])

Epoch 1/200
151/151 [==============================] - 88s 585ms/step - loss: 2.8139 - acc: 0.7126 - val_loss: 1.9973 - val_acc: 0.7814
Epoch 2/200
151/151 [==============================] - 77s 511ms/step - loss: 1.9700 - acc: 0.7893 - val_loss: 1.6640 - val_acc: 0.8406
Epoch 3/200
151/151 [==============================] - 77s 512ms/step - loss: 1.7640 - acc: 0.8150 - val_loss: 1.5068 - val_acc: 0.8532
Epoch 4/200
151/151 [==============================] - 77s 510ms/step - loss: 1.6279 - acc: 0.8339 - val_loss: 1.4362 - val_acc: 0.8586
Epoch 5/200
151/151 [==============================] - 77s 510ms/step - loss: 1.5339 - acc: 0.8435 - val_loss: 1.3562 - val_acc: 0.8719
Epoch 6/200
151/151 [==============================] - 77s 511ms/step - loss: 1.4586 - acc: 0.8551 - val_loss: 1.3207 - val_acc: 0.8884
Epoch 7/200
151/151 [==============================] - 77s 511ms/step - loss: 1.4074 - acc: 0.8627 - val_loss: 1.2627 - val_acc: 0.8919
Epoch 8/200
151/151 [===========================

Exception ignored in: <generator object data_generate2 at 0x7f6184fd8b48>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f6184fd8ba0>
RuntimeError: generator ignored GeneratorExit


Epoch 1/200
151/151 [==============================] - 89s 589ms/step - loss: 2.8173 - acc: 0.7134 - val_loss: 2.0302 - val_acc: 0.7679
Epoch 2/200
151/151 [==============================] - 76s 507ms/step - loss: 2.0010 - acc: 0.7877 - val_loss: 1.6723 - val_acc: 0.8190
Epoch 3/200
151/151 [==============================] - 76s 506ms/step - loss: 1.7964 - acc: 0.8107 - val_loss: 1.5501 - val_acc: 0.8441
Epoch 4/200
151/151 [==============================] - 77s 509ms/step - loss: 1.6465 - acc: 0.8293 - val_loss: 1.4121 - val_acc: 0.8542
Epoch 5/200
151/151 [==============================] - 76s 506ms/step - loss: 1.5324 - acc: 0.8411 - val_loss: 1.3814 - val_acc: 0.8602
Epoch 6/200
151/151 [==============================] - 77s 508ms/step - loss: 1.4710 - acc: 0.8509 - val_loss: 1.3136 - val_acc: 0.8774
Epoch 7/200
151/151 [==============================] - 77s 509ms/step - loss: 1.4106 - acc: 0.8635 - val_loss: 1.2859 - val_acc: 0.8803
Epoch 8/200
151/151 [===========================

Exception ignored in: <generator object data_generate2 at 0x7f60d05937d8>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f6184fd8b48>
RuntimeError: generator ignored GeneratorExit


Epoch 1/200
151/151 [==============================] - 93s 615ms/step - loss: 2.8511 - acc: 0.7096 - val_loss: 2.0189 - val_acc: 0.7688
Epoch 2/200
151/151 [==============================] - 78s 514ms/step - loss: 2.0072 - acc: 0.7830 - val_loss: 1.7125 - val_acc: 0.8287
Epoch 3/200
151/151 [==============================] - 78s 514ms/step - loss: 1.7777 - acc: 0.8091 - val_loss: 1.5745 - val_acc: 0.8459
Epoch 4/200
151/151 [==============================] - 77s 513ms/step - loss: 1.6245 - acc: 0.8294 - val_loss: 1.4806 - val_acc: 0.8608
Epoch 5/200
151/151 [==============================] - 78s 514ms/step - loss: 1.5369 - acc: 0.8408 - val_loss: 1.4235 - val_acc: 0.8669
Epoch 6/200
151/151 [==============================] - 78s 513ms/step - loss: 1.4481 - acc: 0.8519 - val_loss: 1.3754 - val_acc: 0.8785
Epoch 7/200
151/151 [==============================] - 77s 513ms/step - loss: 1.4082 - acc: 0.8603 - val_loss: 1.3369 - val_acc: 0.8855
Epoch 8/200
151/151 [===========================

Exception ignored in: <generator object data_generate2 at 0x7f6055805e60>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f60d05937d8>
RuntimeError: generator ignored GeneratorExit


Epoch 1/200
151/151 [==============================] - 94s 621ms/step - loss: 2.8342 - acc: 0.7062 - val_loss: 2.0013 - val_acc: 0.7787
Epoch 2/200
151/151 [==============================] - 77s 510ms/step - loss: 1.9920 - acc: 0.7856 - val_loss: 1.6821 - val_acc: 0.8293
Epoch 3/200
151/151 [==============================] - 77s 509ms/step - loss: 1.7679 - acc: 0.8106 - val_loss: 1.5354 - val_acc: 0.8451
Epoch 4/200
151/151 [==============================] - 77s 509ms/step - loss: 1.6401 - acc: 0.8301 - val_loss: 1.4492 - val_acc: 0.8608
Epoch 5/200
151/151 [==============================] - 77s 510ms/step - loss: 1.5407 - acc: 0.8437 - val_loss: 1.3842 - val_acc: 0.8714
Epoch 6/200
151/151 [==============================] - 77s 510ms/step - loss: 1.4643 - acc: 0.8513 - val_loss: 1.3346 - val_acc: 0.8782
Epoch 7/200
151/151 [==============================] - 77s 509ms/step - loss: 1.4176 - acc: 0.8606 - val_loss: 1.2748 - val_acc: 0.8917
Epoch 8/200
151/151 [===========================

Exception ignored in: <generator object data_generate2 at 0x7f5f604a7990>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f6055805e60>
RuntimeError: generator ignored GeneratorExit


Epoch 1/200
151/151 [==============================] - 99s 653ms/step - loss: 2.8071 - acc: 0.7077 - val_loss: 1.9850 - val_acc: 0.7735
Epoch 2/200
151/151 [==============================] - 78s 517ms/step - loss: 1.9727 - acc: 0.7873 - val_loss: 1.6937 - val_acc: 0.8293
Epoch 3/200
151/151 [==============================] - 78s 516ms/step - loss: 1.7436 - acc: 0.8130 - val_loss: 1.5880 - val_acc: 0.8432
Epoch 4/200
151/151 [==============================] - 78s 517ms/step - loss: 1.6138 - acc: 0.8278 - val_loss: 1.4673 - val_acc: 0.8590
Epoch 5/200
151/151 [==============================] - 78s 518ms/step - loss: 1.5260 - acc: 0.8442 - val_loss: 1.3957 - val_acc: 0.8717
Epoch 6/200
151/151 [==============================] - 78s 517ms/step - loss: 1.4584 - acc: 0.8542 - val_loss: 1.3607 - val_acc: 0.8767
Epoch 7/200
151/151 [==============================] - 78s 516ms/step - loss: 1.3925 - acc: 0.8644 - val_loss: 1.3582 - val_acc: 0.8843
Epoch 8/200
151/151 [===========================

In [20]:
pred.shape

(4842, 17)

In [21]:
preds = pred / 30

In [22]:
np.sum(preds, axis=1)

array([1.00000001, 0.99999999, 1.        , ..., 0.99999999, 1.        ,
       1.00000001])

In [23]:
result = np.zeros((4842, 17))

In [24]:
for i ,ind in enumerate(np.argmax(preds,axis=1)):
    result[i][ind] = 1

In [25]:
result

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [26]:
import pandas as pd
s = pd.DataFrame(result,dtype='int')

In [27]:
s.to_csv('round2_56-result.csv', header=None, index=False)